# Running Time

In [1]:
!pip install mysql-connector-python redis elasticsearch



In [8]:
import mysql.connector
import redis
from elasticsearch import Elasticsearch

# Connect to MySQL
mysql_conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="mysecret",
    database="testdb"
)
mysql_cursor = mysql_conn.cursor()
mysql_cursor.execute("SELECT VERSION();")
print("MySQL Version:", mysql_cursor.fetchone())

# Connect to Redis
redis_conn = redis.Redis(host='localhost', port=6379, db=0)
redis_conn.set('test', 'Hello Redis')
print("Redis Test:", redis_conn.get('test').decode())

# Connect to Elasticsearch
es = Elasticsearch(["http://localhost:9200"])
es.indices.create(index='test-index', ignore=400)
doc = {"hello": "elasticsearch"}
es.index(index="test-index", id=1, body=doc)
print("Elasticsearch Index Test:", es.get(index="test-index", id=1)['_source'])


MySQL Version: ('8.3.0',)
Redis Test: Hello Redis
Elasticsearch Index Test: {'hello': 'elasticsearch'}


In [16]:
import mysql.connector
import redis
from elasticsearch import Elasticsearch
from time import sleep

def connect_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="mysecret",
        database="testdb"
    )

def connect_redis():
    return redis.Redis(host='localhost', port=6379, db=0)

def connect_elasticsearch():
    return Elasticsearch(["http://localhost:9200"])

# Function to sync data across MySQL, Redis, and Elasticsearch
def sync_data():
    mysql_conn = connect_mysql()
    redis_conn = connect_redis()
    es = connect_elasticsearch()

    cursor = mysql_conn.cursor()
    cursor.execute("SELECT id, username, email FROM users ORDER BY id DESC LIMIT 1;")
    latest_user = cursor.fetchone()
    
    if latest_user:
        user_id, username, email = latest_user
        # Sync to Redis
        redis_conn.set(f"user:{user_id}:username", username)
        redis_conn.set(f"user:{user_id}:email", email)

        # Sync to Elasticsearch
        doc = {
            "username": username,
            "email": email
        }
        es.index(index="users", id=user_id, body=doc)
        print(f"Data synced for user {user_id}")

    cursor.close()
    mysql_conn.close()
sync_data()
# Periodically check for new data
# while True:
#     sync_data()
#     sleep(100)  # sleep for 10 seconds before next check


In [19]:
import time

def benchmark_mysql(cursor):
    start_time = time.perf_counter()
    cursor.execute("INSERT INTO users (username, email) VALUES ('testuser', 'test@example.com')")
    elapsed = time.perf_counter() - start_time
    print(f"MySQL Write Time: {elapsed:.4f} seconds")

    start_time = time.perf_counter()
    cursor.execute("SELECT * FROM users WHERE username = 'testuser'")
    results = cursor.fetchall()
    elapsed = time.perf_counter() - start_time
    print(f"MySQL Read Time: {elapsed:.4f} seconds")

def benchmark_redis(redis_conn):
    start_time = time.perf_counter()
    redis_conn.set("testkey", "testvalue")
    elapsed = time.perf_counter() - start_time
    print(f"Redis Write Time: {elapsed:.4f} seconds")

    start_time = time.perf_counter()
    value = redis_conn.get("testkey")
    elapsed = time.perf_counter() - start_time
    print(f"Redis Read Time: {elapsed:.4f} seconds")

def benchmark_elasticsearch(es):
    doc = {"username": "testuser", "email": "test@example.com"}
    start_time = time.perf_counter()
    es.index(index="users", id=1, body=doc)
    elapsed = time.perf_counter() - start_time
    print(f"Elasticsearch Write Time: {elapsed:.4f} seconds")

    start_time = time.perf_counter()
    result = es.get(index="users", id=1)
    elapsed = time.perf_counter() - start_time
    print(f"Elasticsearch Read Time: {elapsed:.4f} seconds")

mysql_conn = connect_mysql()
cursor = mysql_conn.cursor()
redis_conn = connect_redis()
es = connect_elasticsearch()

benchmark_mysql(cursor)
benchmark_redis(redis_conn)
benchmark_elasticsearch(es)

cursor.close()
mysql_conn.close()


MySQL Write Time: 0.0025 seconds
MySQL Read Time: 0.0017 seconds
Redis Write Time: 0.0059 seconds
Redis Read Time: 0.0008 seconds
Elasticsearch Write Time: 0.0186 seconds
Elasticsearch Read Time: 0.0066 seconds


# Throughput

In [26]:
import faker
fake = faker.Faker()

def generate_user_data(num_records):
    return [{
        'username': fake.user_name(),
        'email': fake.email(),
        'description': fake.text(),
        'created_at': fake.date_time_this_decade().isoformat()
    } for _ in range(num_records)]


In [29]:
import mysql.connector
import redis
from elasticsearch import Elasticsearch
import time

def connect_mysql():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="mysecret",
        database="new_testdb"
    )

def connect_redis():
    return redis.Redis(host='localhost', port=6379, db=0)

def connect_elasticsearch():
    return Elasticsearch(["http://localhost:9200"])

def insert_mysql_data(mysql_conn, data):
    cursor = mysql_conn.cursor()
    start_time = time.perf_counter()
    try:
        for user in data:
            cursor.execute("INSERT INTO users (username, email, description, created_at) VALUES (%s, %s, %s, %s)",
                           (user['username'], user['email'], user['description'], user['created_at']))
        mysql_conn.commit()  # Committing changes directly through the connection object
    except mysql.connector.Error as e:
        print("Error:", e)
    finally:
        cursor.close()  # Make sure to close the cursor here
    elapsed = time.perf_counter() - start_time
    throughput = len(data) / elapsed
    print(f"MySQL Throughput: {throughput:.2f} transactions per second")


def insert_redis_data(redis_conn, data):
    start_time = time.perf_counter()
    for i, user in enumerate(data):
        redis_conn.set(f"user:{i}:username", user['username'])
        redis_conn.set(f"user:{i}:email", user['email'])
    elapsed = time.perf_counter() - start_time
    throughput = len(data) / elapsed
    print(f"Redis Throughput: {throughput:.2f} transactions per second")

def insert_elasticsearch_data(es, data):
    start_time = time.perf_counter()
    for i, user in enumerate(data):
        doc = {'username': user['username'], 'email': user['email'], 'description': user['description'], 'created_at': user['created_at']}
        es.index(index="new_users", id=i, body=doc)
    elapsed = time.perf_counter() - start_time
    throughput = len(data) / elapsed
    print(f"Elasticsearch Throughput: {throughput:.2f} transactions per second")


In [34]:
# Generate data
mysql_conn = connect_mysql()
data = generate_user_data(5000)  # Generate 1000 user records
insert_mysql_data(mysql_conn, data)  # Test MySQL throughput using the connection
mysql_conn.close()  # Close the connection after operations are done


redis_conn = connect_redis()
insert_redis_data(redis_conn, data)  # Test Redis throughput

es = connect_elasticsearch()
insert_elasticsearch_data(es, data)  # Test Elasticsearch throughput


MySQL Throughput: 714.00 transactions per second
Redis Throughput: 487.70 transactions per second
Elasticsearch Throughput: 19.68 transactions per second


# Running Time on NEW DB

In [43]:
def benchmark_mysql_query(mysql_conn):
    cursor = mysql_conn.cursor()
    start_times = []

    # Execute multiple reads to calculate average time
    for _ in range(5000):  # Number of sample queries to average
        start_time = time.perf_counter()
        cursor.execute("SELECT * FROM users WHERE email LIKE '%example.com'")
        _ = cursor.fetchall()
        elapsed = time.perf_counter() - start_time
        start_times.append(elapsed)

    avg_time = sum(start_times) / len(start_times)
    print(f"MySQL Average Query Time: {avg_time:.4f} seconds")
    cursor.close()


In [44]:
def benchmark_redis_query(redis_conn):
    start_times = []

    # Assuming keys are like user:1, user:2, ..., user:100
    for i in range(1, 5001):
        start_time = time.perf_counter()
        _ = redis_conn.get(f"user:{i}:email")  # Direct access if keys are known
        elapsed = time.perf_counter() - start_time
        start_times.append(elapsed)

    avg_time = sum(start_times) / len(start_times)
    print(f"Redis Average Query Time: {avg_time:.4f} seconds")


In [45]:
def benchmark_elasticsearch_query(es):
    start_times = []

    for _ in range(5000):  # Number of sample queries to average
        start_time = time.perf_counter()
        es.search(index="users", body={"query": {"match": {"email": "example.com"}}})
        elapsed = time.perf_counter() - start_time
        start_times.append(elapsed)

    avg_time = sum(start_times) / len(start_times)
    print(f"Elasticsearch Average Query Time: {avg_time:.4f} seconds")


In [46]:
mysql_conn = connect_mysql()
benchmark_mysql_query(mysql_conn)
mysql_conn.close()

redis_conn = connect_redis()
benchmark_redis_query(redis_conn)

es = connect_elasticsearch()
benchmark_elasticsearch_query(es)


MySQL Average Query Time: 0.0163 seconds
Redis Average Query Time: 0.0009 seconds
Elasticsearch Average Query Time: 0.0503 seconds


# Full-Text Searches Running Time

In [57]:
def benchmark_mysql_fulltext_search(mysql_conn):
    cursor = mysql_conn.cursor()
    start_times = []

    # Perform multiple full-text search queries to calculate average time
    for _ in range(5000):  # Adjust the number of queries for better averaging
        query = "SELECT * FROM users WHERE MATCH(description) AGAINST('+data +structures' IN BOOLEAN MODE);"
        start_time = time.perf_counter()
        cursor.execute(query)
        _ = cursor.fetchall()
        elapsed = time.perf_counter() - start_time
        start_times.append(elapsed)

    avg_time = sum(start_times) / len(start_times)
    print(f"ElasticSearch Full-Text Search Average Time: {avg_time:.4f} seconds")
    cursor.close()


In [58]:
def benchmark_elasticsearch_fulltext_search(es):
    start_times = []

    for _ in range(5000):  # Similar number of queries for consistency
        start_time = time.perf_counter()
        es.search(index="users", body={
            "query": {
                "match": {
                    "description": {
                        "query": "data structures",
                        "operator": "and"
                    }
                }
            }
        })
        elapsed = time.perf_counter() - start_time
        start_times.append(elapsed)

    avg_time = sum(start_times) / len(start_times)
    print(f"MySQL Full-Text Search Average Time: {avg_time:.4f} seconds")


In [59]:
mysql_conn = connect_mysql()
benchmark_mysql_fulltext_search(mysql_conn)
mysql_conn.close()

es = connect_elasticsearch()
benchmark_elasticsearch_fulltext_search(es)


ElasticSearch Full-Text Search Average Time: 0.0014 seconds
MySQL Full-Text Search Average Time: 0.0503 seconds
